In [2]:
from pyspark.sql import SparkSession
import psycopg2


In [3]:
spark = SparkSession.builder.appName('provider_new').getOrCreate()

25/04/30 10:36:25 WARN Utils: Your hostname, nischit-baral resolves to a loopback address: 127.0.1.1; using 10.10.42.113 instead (on interface enp2s0)
25/04/30 10:36:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/30 10:36:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

In [5]:
# df_read = spark.read.parquet('in_prov.parquet')
# df_read.printSchema()
df_read = spark.read.parquet('in_net.parquet')
df_read.printSchema()

root
 |-- billing_code: string (nullable = true)
 |-- billing_code_type: string (nullable = true)
 |-- negotiation_arrangement: string (nullable = true)
 |-- billing_class: string (nullable = true)
 |-- negotiated_rate: double (nullable = true)
 |-- billing_code_modifier: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- negotiated_type: string (nullable = true)
 |-- service_code: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- provider_group_id: array (nullable = true)
 |    |-- element: long (containsNull = true)



In [6]:
conn = psycopg2.connect(
    dbname="my_pgdb",
    user="postgres",
    password="admin",
    host="localhost",
    port=5432
)
# conn.autocommit = True

jdbc_url = "jdbc:postgresql://localhost:5432/my_pgdb"
connection_properties = {
    "user": "postgres",
    "password": "admin",
    "driver": "org.postgresql.Driver"
}


25/04/30 10:36:39 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [28]:
# cur = conn.cursor()
# create_table_query = """
# CREATE TABLE IF NOT EXISTS provider1_data (
#     provider_group_id BIGINT,
#     npi BIGINT,
#     tin_type SMALLINT,
#     tin TEXT
# );
# """
# cur.execute(create_table_query)


# df_read.write.jdbc(url=jdbc_url,table="provider1_data",mode="append", properties=connection_properties)



# conn.commit()



In [7]:
cur = conn.cursor()

create_table_query = """

CREATE TABLE IF NOT EXISTS network1_data (
    billing_code TEXT,
    billing_code_type TEXT,
    negotiation_arrangement TEXT,
    provider_group_id BIGINT,
    billing_class TEXT,
    billing_code_modifier TEXT[],
    negotiated_rate DOUBLE PRECISION,
    negotiated_type TEXT,
    service_code INTEGER[]
);
"""
cur.execute(create_table_query)

conn.commit()


In [8]:
df_read.write.jdbc(url=jdbc_url,table="network1_data",mode="append", properties=connection_properties)
conn.commit()

In [ ]:
cur.close()
conn.close()